In [38]:
import torch
import imageio
from PINN import PINN
from Net import Net
import numpy as np
import pandas as pd
import os
import time
import sklearn.mixture as mixture

In [39]:
data_dir = f"{os.getcwd()}/pinn_test_data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")
df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = np.load(data_dir + "/source_points.npy") # shape=(n_source, 3)
sensor_points = np.load(data_dir + "/sensor_points.npy") # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
x_min = df_bounds['x_min'][0]
x_max = df_bounds['x_max'][0]
y_min = df_bounds['y_min'][0]
y_max = df_bounds['y_max'][0]
z_min = df_bounds['z_min'][0]
z_max = df_bounds['z_max'][0]

x_max = 1
y_max = 1
z_max = 1
tfinal = 4.
source_location = np.array([[.3,.3,.3],[.25,.75,.5]])

ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

In [40]:
sigma=.01
model = PINN([30,30,30])
model.set_location(source_location,[tfinal,x_max,y_max,z_max],source_values=[0,.01],sigma=sigma)


c:\Users\m10936927\Documents\methane_project\methane_pinn_dev\methane_pinn_dev-main\Net.py:12: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  torch.nn.init.xavier_uniform(m.weight)


In [41]:
x_max

1

In [42]:
optimizer = torch.optim.Adam(model.net.parameters(), lr=1e-4)
# from torch.optim.lr_scheduler import ExponentialLR

# scheduler = ExponentialLR(optimizer, gamma=0.999)  # Decay LR by 5% every epoch

In [43]:
n= int(5e2)
# n=1
sn = int(5e2)
best_loss = np.inf
max_epochs = int(2e4)
print_freq = 100

sampling_freq = 10 # how often to resample collocation and source points

for epoch in range(max_epochs):
    start_time = time.time()
    if epoch % sampling_freq == 0:
        # points for source PDE loss. uvpoints are wind field at different times. source_values is the value of source at source_colloc_points (assumes Gaussian for now)
        source_collocation_points = model.source_points(sn,2*sigma) 
        # source_uv_points = torch.ones(len(source_collocation_points),2)*.5
        # initial condition collocation points with smaller time values 0.1*t_final. 
        # ic_col = torch.cat([torch.rand(sn,1)*tfinal*.1, torch.rand(sn,1)*x_max, torch.rand(sn,1)*y_max, torch.rand(sn,1)*z_max], dim=1)
        collocation_points = torch.cat([torch.rand(n,1)*tfinal, torch.rand(n,1)*x_max*2 - x_max/2, torch.rand(n,1)*y_max*2 - y_max/2, torch.rand(n,1)*z_max*2-z_max/2], dim=1)
        # collocation_points = torch.cat([collocation_points,ic_col,source_collocation_points])
        collocation_points = torch.cat([collocation_points,source_collocation_points])
        collocation_points.requires_grad_(True)
        # collocation_points.requires_grad=True
        uv = torch.ones(len(collocation_points),2)*.1#wind tensor
        # print(collocation_points[-2])
        uv[:,1:]*= -1
    optimizer.zero_grad()

    loss_1 ,pde_1 = model.compute_pde_loss(collocation_points,uv) # PDE residual loss
    loss_2 = model.compute_negative_loss(collocation_points)
    # loss_2,pde_2 = model.loss_function(source_collocation_points, source_uv_points) # source term PDE residual loss 
    # loss_3 ,pde_3 = model.loss_function(torch.concat([collocation_points,source_collocation_points]),torch.concat([collocation_points,source_collocation_points]))

    # loss = loss_1 + loss_2
    # loss = loss_2*100
    loss = loss_1+loss_2*100

    # # print loss at first epoch
    # if epoch == 0:
    #     print('epoch: %d, loss: %1.3e, pde_res: %1.3e, source_loss: %1.3e' % (epoch, loss.item(), loss_1.item(), loss_2.item()))

    loss.backward()

    # compute norm of gradient of the network
    grad_norm = 0
    for p in model.net.parameters():
        grad_norm += p.grad.data.norm(2).item()**2
    grad_norm = grad_norm**0.5


    if loss.item() < best_loss:
        torch.save(model,'best_mod.m')
    optimizer.step()

    end_time = time.time()
    epoch_time = end_time - start_time
    # scheduler.step()

    if epoch % print_freq == 0:

        # print epoch and loss using %1.3e format
        # print('epoch: %d, loss: %1.3e, grad_norm: %1.3e, pde_res: %1.3e, source_loss: %1.3e, time: %1.3e' % (epoch, loss.item(), grad_norm, loss_1.item(), loss_2.item(), epoch_time))
        print('epoch: %d, loss: %1.3e, grad_norm: %1.3e, pde_res: %1.3e, time: %1.3e' % (epoch, loss.item(), grad_norm, loss_1.item(), epoch_time))

        # print(epoch, loss.item())
        # print(loss_2.item(),loss_1.item())


epoch: 0, loss: 6.529e+03, grad_norm: 5.643e+02, pde_res: 6.529e+03, time: 2.802e-02
epoch: 100, loss: 6.812e+03, grad_norm: 5.474e+02, pde_res: 6.812e+03, time: 2.387e-02
epoch: 200, loss: 8.185e+03, grad_norm: 5.750e+02, pde_res: 8.185e+03, time: 0.000e+00
epoch: 300, loss: 7.662e+03, grad_norm: 5.288e+02, pde_res: 7.662e+03, time: 1.012e-02
epoch: 400, loss: 6.004e+03, grad_norm: 4.236e+02, pde_res: 6.004e+03, time: 1.668e-02
epoch: 500, loss: 6.623e+03, grad_norm: 4.552e+02, pde_res: 6.623e+03, time: 1.000e-03
epoch: 600, loss: 6.032e+03, grad_norm: 2.777e+02, pde_res: 6.032e+03, time: 1.585e-02
epoch: 700, loss: 9.299e+03, grad_norm: 5.082e+02, pde_res: 9.299e+03, time: 1.747e-02
epoch: 800, loss: 6.344e+03, grad_norm: 2.620e+02, pde_res: 6.344e+03, time: 1.562e-02
epoch: 900, loss: 8.761e+03, grad_norm: 3.434e+02, pde_res: 8.761e+03, time: 6.672e-03
epoch: 1000, loss: 6.138e+03, grad_norm: 1.254e+02, pde_res: 6.138e+03, time: 1.561e-02
epoch: 1100, loss: 7.124e+03, grad_norm: 1.8

KeyboardInterrupt: 

In [ ]:
print(x_max)

1


In [ ]:
model = torch.load('best_mod.m')

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL PINN.PINN was not an allowed global by default. Please use `torch.serialization.add_safe_globals([PINN])` or the `torch.serialization.safe_globals([PINN])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch

In [44]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = .5
# net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 100
x_grid = np.linspace(0, x_max, n)
y_grid = np.linspace(0, y_max, n)
z_grid = np.linspace(0, z_max, n)

X, Y, = np.meshgrid(x_grid, y_grid)
Z= X * 0 + Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points, dtype=torch.float32)

time_steps = np.linspace(0, tfinal, 20)  # 10 time steps from 0 to 1


'''
# Plot the concentration over time
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # fix colorbar from 0 to 1 
    # plt.clim(0, 10.0)  # Set colorbar limits
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    # fix colorbar 
    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
'''

'\n# Plot the concentration over time\nfor t in time_steps:\n    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input\n    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape\n    \n    plt.figure()\n    plt.contourf(X, Y, concentration, levels=50, cmap=\'viridis\', vmin=0, vmax=10)  # Plot concentration as a contour plot\n    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), \'ro\', label=\'Source Location\')  # Plot the source location\n    plt.colorbar(label=\'Concentration\')\n    # fix colorbar from 0 to 1 \n    # plt.clim(0, 10.0)  # Set colorbar limits\n    plt.title(f"Gas Concentration at t = {t:.2f}")\n    plt.xlabel(\'x\')\n    plt.ylabel(\'y\')\n    # fix colorbar \n    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image\n'

In [46]:
# save as a GIF
import imageio
import os

images = []
# source_loc = torch.tensor([[.5,.5,.5]])
source_loc = model.source_locs
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = model.forward(torch.cat([t_tensor,grid_points],dim=1),scaled=True).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=100)  # Plot concentration as a contour plot
    plt.plot(model.source_locs[1,0], model.source_locs[1,1], 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'test_visual_basic.gif', images)  # Save the images as a GIF



C:\Users\m10936927\AppData\Local\Temp\ipykernel_16652\3356015117.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list


In [ ]:
print(len(model.net.hidden))

In [ ]:
print(source_points)